In [39]:
import pandas as pd

In [40]:
df_fusionné = pd.read_csv('df_fusionné.csv')

In [41]:
df_fusionné_binaire = df_fusionné.sample(frac=0.2)
df_fusionné_binaire.iloc[:, 0] = df_fusionné_binaire.iloc[:, 0].apply(lambda x: 0 if x == 'N' else 1)

In [42]:
# Count the occurrences of each class
counts = df_fusionné_binaire.iloc[:, 0].value_counts()

# Find the number of instances in the minority class
min_count = counts.min()

# Create balanced DataFrame through undersampling
df_balanced_under = pd.concat([
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 0].sample(min_count),
    df_fusionné_binaire[df_fusionné_binaire.iloc[:, 0] == 1].sample(min_count)
])

# Shuffle the DataFrame to mix the classes
df_balanced_under = df_balanced_under.sample(frac=1, random_state=42)


In [43]:
# Count occurrences of 0 and 1 in the first column
value_counts = df_balanced_under.iloc[:, 0].value_counts()

# Display the counts
print(value_counts)


0    7566
1    7566
Name: ColumnName, dtype: int64


In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = df_balanced_under.iloc[:, 1:]  
y = df_balanced_under.iloc[:, 0] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
print(y_train.unique())


[0 1]


In [46]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {
    "criterion": ["gini", "entropy"],  # 'log_loss' retiré
    "splitter": ["best", "random"],
    "max_depth": [4, 7, 12],
}


base_model = DecisionTreeClassifier()
model = GridSearchCV(estimator=base_model, param_grid=params, cv=4)

# Ajustez le modèle
model.fit(X_train, y_train)

# Maintenant, utilisez model.best_estimator_ pour faire des prédictions
y_pred = model.best_estimator_.predict(X_test)


In [59]:
best_params = model.best_params_
best_params


{'criterion': 'gini', 'max_depth': 12, 'splitter': 'random'}

In [49]:
#model.get_params()

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6891311529567228

In [63]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred, labels=[0,1])

array([[1156,  350],
       [ 591,  930]], dtype=int64)

In [64]:
from sklearn.metrics import precision_score
precision_score(y_test, y_pred)

0.7265625

In [65]:
from sklearn.metrics import recall_score
recall_score(y_test, y_pred)

0.611439842209073

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=['malignant', 'benign']))

              precision    recall  f1-score   support

   malignant       0.66      0.77      0.71      1506
      benign       0.73      0.61      0.66      1521

    accuracy                           0.69      3027
   macro avg       0.69      0.69      0.69      3027
weighted avg       0.69      0.69      0.69      3027



In [67]:
feature_names = X.columns
feature_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '350', '351', '352', '353', '354', '355', '356', '357', '358', '359'],
      dtype='object', length=360)